In [15]:
# what can python tell the Princeton Python Meetup about the upcoming eclipse?
# Princeton Python Meetup: https://www.meetup.com/pug-ip/
# pug-ip is "python user group - in princeton"

# [install steps here!]  <---------- !!!


In [16]:
# where is the observer?

import requests
# requests is not built into python but is very popular
# http://docs.python-requests.org/en/master/user/quickstart/

def getLatLon(streetAddr, apiKey = None):
    '''return lat,lon from street address string
        google is more accurate (to the street number) but requires a (free as in beer) key
        openstreetmap is missing our street number but knows our street
        '''
    gglTemplate = 'https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s'
    osmTemplate = 'http://nominatim.openstreetmap.org/search?q=%s&format=json'
    if apiKey:
        url = gglTemplate % (streetAddr, apiKey)
    else:
        url = osmTemplate % streetAddr
    r = response = requests.get(url)
    if not r.status_code == requests.codes.ok:
        r.raise_for_status()
    if apiKey:
        # no license embedded, but agreed to before receipt of api key
        return Loc(**r.json()['results'][0]['geometry']['location'])
    else:
        data = r.json()[0]
        # print(data)  # note license in data['licence'] (non-American spelling)
        return dict(lat = data['lat'], lon = data['lon'])

tigerLabsLocation = getLatLon('252 Nassau St, Princeton NJ')
print(tigerLabsLocation)
# {'lat': '40.3491234', 'lon': '-74.6617155'}
# this is just a few blocks away: plenty close enough for astronomy!
# see map by dropping "&format=json" from osm url in browser.
# OSM has 246 Nassau but not 252 Nassau.


{'lon': '-74.6617155', 'lat': '40.3491234'}


In [17]:
# on Aug 21, is there a midafternoon eclipse here?

# pythonistas do astronomy with pyephem
# http://rhodesmill.org/pyephem/
import ephem
from math import pi

def createObserverAt(location, date = None):
    obs = ephem.Observer()
    obs.lat = location['lat']
    obs.lon = location['lon']
    if date:
        obs.date = ephem.Date(date)
    return obs
    
sun = ephem.Sun()
moon = ephem.Moon()

# concept: Observer = when + where; when defaults to now
pugipNow = createObserverAt(tigerLabsLocation)
# default date is now
print('now in localtime',ephem.localtime(pugipNow.date).ctime())
# dates default to UTC
print('now in UTC',pugipNow.date)

secondsOfArcPerRadian = (180 / pi) * 60 * 60
# an eclipse is when moon-sun distance is less than the sum of their radii
def isEclipse(m, s, obs):
    m.compute(obs)
    s.compute(obs)
    sep = ephem.separation((m.az, m.alt), (s.az, s.alt))
    sep = sep * secondsOfArcPerRadian
    radii = m.size / 2. + s.size / 2.  # already in seconds of arc
    return sep < radii

# is there an eclipse right now?
answer = 'YES' if isEclipse(moon, sun, pugipNow) else 'no'
print('is there an eclipse right now?',answer)

# is there an eclipse on 21aug at 2pm ET?
duringEclipse = '2017/8/21 18:00:00'  # approximate
pugipDuringEclipse = createObserverAt(tigerLabsLocation, duringEclipse)
#print(ephem.localtime(date).ctime())
answer = 'YES' if isEclipse(moon, sun, pugipDuringEclipse) else 'no'
print('is there an eclipse at 2pm on 21aug?',answer)


now in localtime Tue Aug 15 06:04:25 2017
now in UTC 2017/8/15 10:04:25
is there an eclipse right now? no
is there an eclipse at 2pm on 21aug? YES


In [18]:
# how eclipsed is the sun by the moon at a given time?
# a convenient measure of eclipse-ness is occlusion or overlap
# how do we compute the overlap of two circles?

# shapely is a popular geometry library
# sudo apt install libgeos-dev libgeos++-dev
# sudo pip install shapely

# in shapely, a circle is a point with a radius 'buffer'
from shapely.geometry import Point

def proportionOverlap(m, s, obs):
    m.compute(obs)
    s.compute(obs)
    secondsOfArcPerRadian = (180 / pi) * 60 * 60  # todo how share btw cells?
    #print('moon',float(m.az),float(m.alt),m.size / 2. / secondsOfArcPerRadian)
    #print('sun',float(s.az),float(s.alt),s.size / 2. / secondsOfArcPerRadian)
    #circle = lambda x: Point(float(x.az),float(x.alt)).buffer(x.size / 2. / secondsOfArcPerRadian)
    circle = lambda x: Point(float(x.ra),float(x.dec)).buffer(x.size / 2. / secondsOfArcPerRadian)
    #circle = lambda x: Point(float(x.g_ra),float(x.g_dec)).buffer(x.size / 2. / secondsOfArcPerRadian)
    overlap = circle(m).intersection(circle(s)).area
    return overlap / circle(s).area

print('overlap now:',proportionOverlap(moon, sun, pugipNow))
print('overlap sometime during eclipse:',proportionOverlap(moon, sun, pugipDuringEclipse))


overlap now: 0.0
overlap sometime during eclipse: 0.30309822942311443


In [19]:
# comic relief: we've found that the moon overlaps the sun, 
#   but which is frontmost and which is obscured?

moon2 = ephem.Moon()
try:
    print(moon2.earth_distance)
except Exception as e:
    print('oops:',e)
# ".earth_distance" requires observer, 
#    so it gives the distance at the observer's time

# distance given in AU or astronomical units
moon.compute(pugipNow)
sun.compute(pugipNow)
print('distance now to: sun',sun.earth_distance,'; moon',moon.earth_distance)

# interesting that the sun is farther than average right now [mid-August]
#   in the northern hemisphere's summer
# exercise for the reader: when is the sun farthest from earth?

# let's check that the moon's distance does indeed vary with date
moon.compute(pugipNow)
moon2earthNow = moon.earth_distance
print('distance to moon now:',moon.earth_distance)
moon.compute(pugipDuringEclipse)
moon2earthDuringEclipse = moon.earth_distance
print('distance to moon duringEclipse:',moon.earth_distance)
assert moon2earthNow != moon2earthDuringEclipse

# finally, which is closer during the eclipse?
if moon.earth_distance < sun.earth_distance:
    print('moon obscures sun :)')
else:
    print('sun obscures moon')


oops: field earth_distance undefined until first compute()
distance now to: sun 1.012747883796692 ; moon 0.002439310075715184
distance to moon now: 0.002439310075715184
distance to moon duringEclipse: 0.002450399100780487
moon obscures sun :)


In [20]:
# when is the best time to view the eclipse?

noon = ephem.Date('2017-08-21 16:00:00')  # UTC
#print('noon',ephem.localtime(noon))  # close enough
hoursPerDay = 24
def hoursSinceNoon(ephemDate):
    return (ephemDate - noon) * hoursPerDay

timeDuringEclipse = ephem.Date(duringEclipse)
safeOffset = 12 * ephem.hour
startTime = timeDuringEclipse - safeOffset
endTime   = timeDuringEclipse + safeOffset
pugip = createObserverAt(tigerLabsLocation, startTime)

data = []

while pugip.date < endTime:
    pugip.date += ephem.minute * 5.
    p = proportionOverlap(moon, sun, pugip)
    if p > 0:
        print(ephem.localtime(pugip.date), p)
        data.append((hoursSinceNoon(pugip.date), p))

# eclipse is 1:25pm - 2:55pm ET
# let's increase resolution near the maximum
pugip.date = ephem.Date('2017/8/21 18:40:00')  # UTC
endTime = pugip.date + 10. * ephem.minute
print()
while pugip.date < endTime:
    pugip.date += 1. * ephem.minute
    p = proportionOverlap(moon, sun, pugip)
    print(ephem.localtime(pugip.date), p)
    data.append((hoursSinceNoon(pugip.date), p))
    
# more, more!
pugip.date = ephem.Date('2017/8/21 18:44:00')  # UTC
endTime = pugip.date + 2. * ephem.minute
print()
while pugip.date < endTime:
    pugip.date += 5. * ephem.second
    p = proportionOverlap(moon, sun, pugip)
    print(ephem.localtime(pugip.date), p)
    data.append((hoursSinceNoon(pugip.date), p))

# max overlap [here] is 74% at 14:44:44 ET

2017-08-21 13:24:59 0.001694107284564178
2017-08-21 13:29:59 0.022510617905891008
2017-08-21 13:34:59 0.054807727390063364
2017-08-21 13:39:59 0.09481706652618094
2017-08-21 13:44:59 0.14072852431040966
2017-08-21 13:49:59 0.1913400777684003
2017-08-21 13:54:59 0.24574093139811679
2017-08-21 13:59:59 0.30309822157773364
2017-08-21 14:04:59 0.3626283697863124
2017-08-21 14:09:59 0.4234714303358568
2017-08-21 14:14:59 0.4846186901303971
2017-08-21 14:19:59 0.5447697599578196
2017-08-21 14:24:59 0.6021169745457123
2017-08-21 14:29:59 0.6540582828556148
2017-08-21 14:34:59 0.6968072969289318
2017-08-21 14:39:59 0.7254318946603809
2017-08-21 14:44:59 0.7349944145927305
2017-08-21 14:49:59 0.7232447022659445
2017-08-21 14:54:59 0.6922706044046211
2017-08-21 14:59:59 0.6468784556333882
2017-08-21 15:04:59 0.591896862613738
2017-08-21 15:09:59 0.5310909683699548
2017-08-21 15:14:59 0.4671264753318823
2017-08-21 15:19:59 0.4018879382779289
2017-08-21 15:24:59 0.33683087577359005
2017-08-21 15:2

In [21]:
# graph and animate these lists of numbers
import bqplot.pyplot as p
data = sorted(data)
xs, ys = zip(*data)

#print(xs[:5], ys[:5])
#print([x for x in xs])

p.figure('overlap')
p.plot(xs, ys)
p.show()

In [22]:
# how cloudy will it be here?
# let's ask the National Weather Service, weather.gov,
#   as I do at http://websitedeli.com/wsgi/fcst/?loc=08540
import requests
urlTemplate = 'http://forecast.weather.gov/MapClick.php?lat={lat}&lon={lon}&FcstType=digitalDWML'
url = urlTemplate.format(**tigerLabsLocation)
r = response = requests.get(url)
if not r.status_code == requests.codes.ok:
    r.raise_for_status()
xml = r.content
#print(xml[:44])

from lxml import etree
import io
tree = etree.parse(io.BytesIO(xml))
from dateutil.parser import parse as parseDate
starttimes=[parseDate(starttime) 
            for starttime in tree.xpath('data/time-layout/start-valid-time/text()')]
#print(starttimes[:5])
#els=[el for el in tree.xpath('data/parameters/node()') if type(el)==etree._Element]
#print([el.tag for el in els])
#cloudinessNode = tree.xpath('data/parameters/cloud-amount')[0]
#print(list(cloudinessNode))
#v=list(cloudinessNode)[24]
#print(v.text)
cloudiness = [node.text for node in tree.xpath('data/parameters/cloud-amount')[0]]
#print(cloudiness)
from pprint import pprint
#print(dir(noon))
import pytz
from datetime import timedelta
utc=pytz.UTC
noonDt=utc.localize(noon.datetime())
fivePM=utc.localize(noon.datetime() + timedelta(0, 5 * 60 * 60))
#print(noon,noonDt)
pprint(list(z for z in zip(starttimes,cloudiness) if noonDt < z[0] < fivePM))
# RESUME

[(datetime.datetime(2017, 8, 21, 12, 0, tzinfo=tzoffset(None, -14400)), '29'),
 (datetime.datetime(2017, 8, 21, 13, 0, tzinfo=tzoffset(None, -14400)), '31'),
 (datetime.datetime(2017, 8, 21, 14, 0, tzinfo=tzoffset(None, -14400)), '32'),
 (datetime.datetime(2017, 8, 21, 15, 0, tzinfo=tzoffset(None, -14400)), '32'),
 (datetime.datetime(2017, 8, 21, 16, 0, tzinfo=tzoffset(None, -14400)), '32')]


In [23]:
# todo
# pinhole
#   crescents under tree
#   light walk--may change the way you think about light and vision
#     https://www.exploratorium.edu/bob-miller/light-walk
#     the exploratorium led sci
# next eclipse: 08apr2024  https://eclipse.gsfc.nasa.gov/SEgoogle/SEgoogle2001/SE2024Apr08Tgoogle.html



In [24]:
# loose ends

# can a jupyter notebook run in a virtualenv so we can pip install without sudo access?

# understand better:
# Apparent geocentric position: g_ra and g_dec: where the object will "appear" for an observer at the center of the Earth
# Apparent topocentric position: ra and dec: apparent position of the body as seen by the Observer on the surface of the Earth
